# **Assignment 3: Pandas**
- **you will learn:** how to create and manipulate Pandas series and dataframes, perform vectorized computations and work with dates and times
- **task:**  See section 3.4 below
- **deadline:** 03.11.2025
- [Pandas documentation](https://pandas.pydata.org/docs/index.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.

## **3.1 Introduction: Connection to NumPy**

The **Pandas** library is a high-level data manipulation tool built on top of **NumPy**.  
While NumPy provides powerful *numerical* operations on arrays, Pandas adds tools for handling **labeled**, **heterogeneous**, and **tabular data** — similar to how you might work with data in an Excel sheet or SQL table.

- NumPy → works with arrays of numbers  
- Pandas → works with **Series** (1D labeled data) and **DataFrames** (2D labeled data)

You can think of **Pandas** as an enhanced version of NumPy — it builds upon its speed and efficiency, but adds **labels**, **indexing**, and **data handling** capabilities that make real-world data analysis much easier.

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
from PIL.ImageChops import difference
from unicodedata import category, numeric

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 2.3.3
Pandas version: 2.3.3


## **3.2 Series**

There are two main data structures in Pandas:

1. **Series** – a one-dimensional labeled array (like a single column)
2. **DataFrame** – a two-dimensional labeled data structure (like a table)

We begin by inspecting the **Series** data structure: a sequence of values with an associated index (which makes it different from a NumPy array).

In [2]:
print("Series comes with data, index and possibly a name. \n")

# 1️⃣ Floats with default index
s1 = pd.Series([1.2, 3.4, 5.6, 7.8])
print("1. Floats with default index:\n", s1, "\n")

# 2️⃣ Floats with explicit string index
s2 = pd.Series([2.5, 3.8, 4.1], index=['a', 'b', 'c'])
print("2. Floats with string index:\n", s2, "\n")

# 3️⃣ Floats with consecutive date index
s3 = pd.Series([10.1, 11.2, 12.3, 13.4], index=pd.date_range("2025-01-01", periods=4))
print("3. Floats with date index:\n", s3, "\n")

# 4️⃣ Mixed data types (int, float, string, bool)
s4 = pd.Series([10, 3.14, 'hello', True], name='A bunch of data...')
print("4. Mixed data types:\n", s4, "\n")

# 5️⃣ Data containing missing values (NaN)
s5 = pd.Series([1.1, np.nan, 2.2, None, 3.3])
print("5. Data with missing values:\n", s5)

# 6️⃣ Data created from NumPy array
arr = np.array([100, 200, 300])
s6 = pd.Series(arr)
print("\n6. Data from NumPy array:\n", s6)

Series comes with data, index and possibly a name. 

1. Floats with default index:
 0    1.2
1    3.4
2    5.6
3    7.8
dtype: float64 

2. Floats with string index:
 a    2.5
b    3.8
c    4.1
dtype: float64 

3. Floats with date index:
 2025-01-01    10.1
2025-01-02    11.2
2025-01-03    12.3
2025-01-04    13.4
Freq: D, dtype: float64 

4. Mixed data types:
 0       10
1     3.14
2    hello
3     True
Name: A bunch of data..., dtype: object 

5. Data with missing values:
 0    1.1
1    NaN
2    2.2
3    NaN
4    3.3
dtype: float64

6. Data from NumPy array:
 0    100
1    200
2    300
dtype: int64


Also the index of a Series has a specific data type, which can vary based on how the Series was created.

In [3]:
s1.index, s2.index, s3.index, s4.index, s5.index, s6.index

(RangeIndex(start=0, stop=4, step=1),
 Index(['a', 'b', 'c'], dtype='object'),
 DatetimeIndex(['2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04'], dtype='datetime64[ns]', freq='D'),
 RangeIndex(start=0, stop=4, step=1),
 RangeIndex(start=0, stop=5, step=1),
 RangeIndex(start=0, stop=3, step=1))

In [4]:
s1[2], s2['b'], s3[pd.Timestamp('2025-01-02')], s4[3], s5[1], s6[len(s6)-1]

(np.float64(5.6),
 np.float64(3.8),
 np.float64(11.2),
 True,
 np.float64(nan),
 np.int64(300))

In [5]:
print("Even with non-integer indices, we can access elements using .iloc:\n")
s3.iloc[2]

Even with non-integer indices, we can access elements using .iloc:



np.float64(12.3)

In [6]:
s2[['a', 'c']]

a    2.5
c    4.1
dtype: float64

In [7]:
print("Slicing works similarly to NumPy arrays:\n")
s1[1:-1]

Slicing works similarly to NumPy arrays:



1    3.4
2    5.6
dtype: float64

In [8]:
print("For non-integer indices, slicing works even with interval boundaries:\n")
s3['2025-01-02':'2025-01-04'], s3[3:5]

For non-integer indices, slicing works even with interval boundaries:



(2025-01-02    11.2
 2025-01-03    12.3
 2025-01-04    13.4
 Freq: D, dtype: float64,
 2025-01-04    13.4
 Freq: D, dtype: float64)

In [9]:
print("Warning: index uniqueness is not enforced, so duplicate indices are possible:\n")
s_dup = pd.Series([10, 20, 30], index=['a', 'b', 'a'])
print(s_dup['a'])  # Returns both values for index 'a'


a    10
a    30
dtype: int64


In [10]:
print("Boolean indexing:\n")
pd.Series([1, 2, 3])[[True, False, True]]

Boolean indexing:



0    1
2    3
dtype: int64

We have touched  **DatetimeIndex** and **Timestamp**, two examples of a long list of specialized pandas objects for date and time manipulation, see the [documentation](https://pandas.pydata.org/docs/user_guide/timeseries.html) for more details.

### **3.2.1 Series Operations**

Pandas Series support a variety of operations similar to NumPy arrays, including arithmetic operations, aggregation functions, and element-wise operations.

In [11]:
s1 = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
s2 = pd.Series([4., 5., 6.], index=['a', 'b', 'c'])

print(s1 + s2)   # Addition
print(s1 - s2)   # Subtraction
print(s1 * s2)   # Multiplication
print(s1 / s2)   # Division

a    5.0
b    7.0
c    9.0
dtype: float64
a   -3.0
b   -3.0
c   -3.0
dtype: float64
a     4.0
b    10.0
c    18.0
dtype: float64
a    0.25
b    0.40
c    0.50
dtype: float64


In [12]:
print("Operations align on index labels:\n")
s1 = pd.Series([1, 2, 3], index=['a', 'b', 'd'])
s2 = pd.Series([4., 5., 6.], index=['a', 'b', 'c'])

print(s1 + s2)   # Addition
print(s1 - s2)   # Subtraction
print(s1 * s2)   # Multiplication
print(s1 / s2)   # Division

Operations align on index labels:

a    5.0
b    7.0
c    NaN
d    NaN
dtype: float64
a   -3.0
b   -3.0
c    NaN
d    NaN
dtype: float64
a     4.0
b    10.0
c     NaN
d     NaN
dtype: float64
a    0.25
b    0.40
c     NaN
d     NaN
dtype: float64


In [13]:
print("Comparison operations:\n")
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
print(s > 2)
print(s == 2)

Comparison operations:

a    False
b    False
c     True
dtype: bool
a    False
b     True
c    False
dtype: bool


In [14]:
print("Aggregation functions:\n")
s = pd.Series([1, 2, 3, 4, 5])
print(s.sum())     # 15
print(s.mean())    # 3.0
print(s.median())  # 3.0
print(s.min())     # 1
print(s.max())     # 5
print(s.std())     # Standard deviation

Aggregation functions:

15
3.0
3.0
1
5
1.5811388300841898


In [15]:
print("Element-wise logical operations:\n")
s1 = pd.Series([True, False, True])
s2 = pd.Series([False, True, True])

print(s1 & s2)  # Element-wise AND
print(s1 | s2)  # Element-wise OR
print(~s1)      # NOT

Element-wise logical operations:

0    False
1    False
2     True
dtype: bool
0    True
1    True
2    True
dtype: bool
0    False
1     True
2    False
dtype: bool


In [16]:
print("String operations on Series of strings:\n")
s = pd.Series(['apple', 'banana', 'cherry'])
print(s.str.upper())   # ['APPLE', 'BANANA', 'CHERRY']
print(s.str.contains('a'))  # [True, True, False]
print(s.str.len())     # [5, 6, 6]

String operations on Series of strings:

0     APPLE
1    BANANA
2    CHERRY
dtype: object
0     True
1     True
2    False
dtype: bool
0    5
1    6
2    6
dtype: int64


In [17]:
print("Also NumPy functions work on Series:\n")
s = pd.Series([1, 4, 9, 16])
print(np.sqrt(s))      # Square root
print(np.log(s))       # Natural logarithm
print(np.exp(s))       # Exponential

Also NumPy functions work on Series:

0    1.0
1    2.0
2    3.0
3    4.0
dtype: float64
0    0.000000
1    1.386294
2    2.197225
3    2.772589
dtype: float64
0    2.718282e+00
1    5.459815e+01
2    8.103084e+03
3    8.886111e+06
dtype: float64


In [18]:
print("Cumulative operations:\n")
s = pd.Series([1, 2, 3, 4, 5])
print(s.cumsum())  # Cumulative sum
print(s.cumprod()) # Cumulative product
print(s.cummax())  # Cumulative maximum
print(s.cummin())  # Cumulative minimum

Cumulative operations:

0     1
1     3
2     6
3    10
4    15
dtype: int64
0      1
1      2
2      6
3     24
4    120
dtype: int64
0    1
1    2
2    3
3    4
4    5
dtype: int64
0    1
1    1
2    1
3    1
4    1
dtype: int64


In [19]:
print("Vectorized operations are efficient:\n")
s = pd.Series(np.arange(1, 1000001))

squared = s.apply(lambda x: x ** 2)
print(squared)

Vectorized operations are efficient:

0                     1
1                     4
2                     9
3                    16
4                    25
              ...      
999995     999992000016
999996     999994000009
999997     999996000004
999998     999998000001
999999    1000000000000
Length: 1000000, dtype: int64


In [20]:
def add_number(x, n):
    return x + n

# Using apply with a lambda to pass the extra argument
added = s.apply(lambda x: add_number(x, 10))
print(added)

0              11
1              12
2              13
3              14
4              15
           ...   
999995    1000006
999996    1000007
999997    1000008
999998    1000009
999999    1000010
Length: 1000000, dtype: int64


## **3.3 DataFrames**

In data analysis, we often work with tabular data consisting of rows and columns, where similarly to an excel spreadsheet, each row represents a record and each column represents a feature or attribute of that record. DataFrames are the primary data structure in Pandas for handling such tabular data.

In [21]:
# ---------------------------
# 1. Creating DataFrame from a NumPy array
# ---------------------------
# NumPy array
array_data = np.array([[10, 20, 30],
                       [40, 50, 60],
                       [70, 80, 90]])

# Create DataFrame
df_from_array = pd.DataFrame(array_data, columns=['A', 'B', 'C'])
print("DataFrame from NumPy array:\n", df_from_array)

# ---------------------------
# 2. Creating DataFrame from a Pandas Series
# ---------------------------
# Series
s1 = pd.Series([100, 200, 300], name='X')
s2 = pd.Series([400, 500, 600], name='Y')

# Combine Series into DataFrame
df_from_series = pd.concat([s1, s2], axis=1)  # warning: compare to the case with axis=0
print("\nDataFrame from Series:\n", df_from_series)

# ---------------------------
# 3. Creating DataFrame from a dictionary
# ---------------------------
# Dictionary
dict_data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 22],
    'City': ['New York', 'Los Angeles', 'Chicago']
}

df_from_dict = pd.DataFrame(dict_data)
print("\nDataFrame from Dictionary:\n", df_from_dict)

# ---------------------------
# 4. Creating DataFrame from a csv file
# ---------------------------

df_from_csv = pd.read_csv('data/csv_example.csv') # Assuming a CSV file 'csv_example.csv' exists in the current 'data' directory
print("\nDataFrame from CSV file:\n", df_from_csv, sep=',')

DataFrame from NumPy array:
     A   B   C
0  10  20  30
1  40  50  60
2  70  80  90

DataFrame from Series:
      X    Y
0  100  400
1  200  500
2  300  600

DataFrame from Dictionary:
       Name  Age         City
0    Alice   25     New York
1      Bob   30  Los Angeles
2  Charlie   22      Chicago

DataFrame from CSV file:
,   EmployeeID             Name Department  Age  Salary JoiningDate FullTime
0         101       John Smith      Sales   28   50000  2019-03-15      Yes
1         102         Jane Doe  Marketing   34   60000  2018-07-23      Yes
2         103     Mike Johnson         IT   30   55000  2020-01-10       No
3         104      Emily Davis         HR   26   48000  2021-06-12      Yes
4         105    William Brown         IT   40   75000  2017-11-03      Yes
5         106     Linda Wilson      Sales   32   52000  2019-08-19       No
6         107        David Lee  Marketing   29   58000  2020-02-25      Yes
7         108        Sarah Kim         HR   35   62000  2016-0

In [22]:
df_from_csv.shape, df_from_csv.columns, df_from_csv.index, df_from_csv.dtypes

((10, 7),
 Index(['EmployeeID', 'Name', 'Department', 'Age', 'Salary', 'JoiningDate',
        'FullTime'],
       dtype='object'),
 RangeIndex(start=0, stop=10, step=1),
 EmployeeID      int64
 Name           object
 Department     object
 Age             int64
 Salary          int64
 JoiningDate    object
 FullTime       object
 dtype: object)

In [23]:
df_from_csv = df_from_csv.astype({'Salary': float})
df_from_csv.dtypes

EmployeeID       int64
Name            object
Department      object
Age              int64
Salary         float64
JoiningDate     object
FullTime        object
dtype: object

In [24]:
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [24, 27, 22, 32, 29],
    'Major': ['Math', 'Physics', 'Biology', 'CS', 'Chemistry'],
    'GPA': [3.5, 3.8, 3.2, 3.9, 3.7]
}

df = pd.DataFrame(data)
print("Original DataFrame:\n", df)

# ---------------------------
# 2. Setting an index
# ---------------------------
df.set_index('Name', inplace=True)
print("\nDataFrame with 'Name' as index:\n", df)

# ---------------------------
# 3. Slicing rows
# ---------------------------
# Using loc (label-based)
print("\nSlice using loc (Alice to Charlie):\n", df.loc['Alice':'Charlie'])

# Using iloc (position-based)
print("\nSlice using iloc (first 3 rows):\n", df.iloc[0:3])

# ---------------------------
# 4. Slicing columns
# ---------------------------
# Select a single column
print("\nSelect 'Age' column:\n", df['Age'])

# Select multiple columns
print("\nSelect 'Age' and 'GPA' columns:\n", df[['Age', 'GPA']])

# Select columns using loc (rows + columns)
print("\nSelect rows Alice to Charlie and columns Age & GPA:\n", df.loc['Alice':'Charlie', ['Age', 'GPA']])

# Select columns using iloc (by positions)
# Rows 0 to 2, columns 0 to 1 (Age and Major)
print("\nSelect first 3 rows and first 2 columns using iloc:\n", df.iloc[0:3, 0:2])

# ---------------------------
# 5. Filtering rows
# ---------------------------
# Filter students with GPA > 3.5
high_gpa = df[df['GPA'] > 3.5]
print("\nStudents with GPA > 3.5:\n", high_gpa)

# Filter students majoring in CS or Physics
selected_majors = df[df['Major'].isin(['CS', 'Physics'])]
print("\nStudents majoring in CS or Physics:\n", selected_majors)

# ---------------------------
# 6. Changing values
# ---------------------------
# Change a single value using loc
df.loc['Alice', 'GPA'] = 3.6
print("\nAfter changing Alice's GPA to 3.6:\n", df)

# Change multiple values using condition
df.loc[df['Major'] == 'CS', 'GPA'] = 4.0
print("\nAfter setting GPA=4.0 for CS majors:\n", df)

# Change an entire column
df['Age'] = df['Age'] + 1  # increment age by 1
print("\nAfter incrementing Age by 1:\n", df)

Original DataFrame:
       Name  Age      Major  GPA
0    Alice   24       Math  3.5
1      Bob   27    Physics  3.8
2  Charlie   22    Biology  3.2
3    David   32         CS  3.9
4      Eva   29  Chemistry  3.7

DataFrame with 'Name' as index:
          Age      Major  GPA
Name                        
Alice     24       Math  3.5
Bob       27    Physics  3.8
Charlie   22    Biology  3.2
David     32         CS  3.9
Eva       29  Chemistry  3.7

Slice using loc (Alice to Charlie):
          Age    Major  GPA
Name                      
Alice     24     Math  3.5
Bob       27  Physics  3.8
Charlie   22  Biology  3.2

Slice using iloc (first 3 rows):
          Age    Major  GPA
Name                      
Alice     24     Math  3.5
Bob       27  Physics  3.8
Charlie   22  Biology  3.2

Select 'Age' column:
 Name
Alice      24
Bob        27
Charlie    22
David      32
Eva        29
Name: Age, dtype: int64

Select 'Age' and 'GPA' columns:
          Age  GPA
Name             
Alice     24  3

In [25]:
print("When updating values using different dataframe, we must be careful.")
df1 = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': [100, 200, 300, 400, 500]
})

df2 = pd.DataFrame({
    'A': [9, 8, 7, 6, 5],
    'B': [90, 80, 70, 60, 50],
    'C': [900, 800, 700, 600, 500]
})

# Without .values
df1.loc[1:2, ['B', 'C']] = df2.loc[3:4, ['B', 'C']]  # it searches for index match in this case
print(df1)

When updating values using different dataframe, we must be careful.
     A     B      C
0  1.0  10.0  100.0
1  2.0   NaN    NaN
2  3.0   NaN    NaN
3  4.0  40.0  400.0
4  5.0  50.0  500.0


C:\Users\x\AppData\Local\Temp\ipykernel_5224\1280378109.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[nan nan]
 [nan nan]]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.loc[1:2, ['B', 'C']] = df2.loc[3:4, ['B', 'C']]  # it searches for index match in this case


In [26]:
df1.loc[1:2, ['B', 'C']] = df2.loc[1:2, ['B', 'C']]
print(df1)

     A     B      C
0  1.0  10.0  100.0
1  2.0  80.0  800.0
2  3.0  70.0  700.0
3  4.0  40.0  400.0
4  5.0  50.0  500.0


In [27]:
df1.loc[1:2, ['B', 'C']] = df2.loc[3:4, ['B', 'C']].values
print(df1)

     A     B      C
0  1.0  10.0  100.0
1  2.0  60.0  600.0
2  3.0  50.0  500.0
3  4.0  40.0  400.0
4  5.0  50.0  500.0


Another major topic is grouping and aggregation in DataFrames, which allows us to perform operations on subsets of data based on certain criteria. This topic will be covered in more detail in later assignments.

## **3.3.1 Inplace vs non-inplace**

In [28]:
data = {
    'Name': ['Alice', 'Bob', None, 'David'],
    'Age': [25, np.nan, 30, np.nan]
}

df = pd.DataFrame(data)
print(df)

    Name   Age
0  Alice  25.0
1    Bob   NaN
2   None  30.0
3  David   NaN


In [29]:
new_df = df.dropna(subset=['Name'])
print(new_df)  # New df without NaN
print(df)      # Original df unchanged

    Name   Age
0  Alice  25.0
1    Bob   NaN
3  David   NaN
    Name   Age
0  Alice  25.0
1    Bob   NaN
2   None  30.0
3  David   NaN


In [30]:
df.dropna(subset=['Name'], inplace=True)
print(df)

    Name   Age
0  Alice  25.0
1    Bob   NaN
3  David   NaN


In [31]:
print("Warning: inplace=True modifies the original DataFrame and returns None, chaining the operations is therefore impossible.\n")
#type(df.dropna(subset=['Name'], inplace=True).fillna(0))

## **3.3.2 Transformations**

In [32]:
# Sample DataFrame
df = pd.DataFrame({
    'Product': ['A', 'B', 'C', 'D'],
    'Price': [100, 200, 150, 300],
    'Quantity': [5, 2, 7, 3]
})

# 1. Adding new columns
df['Total'] = df['Price'] * df['Quantity']

# 2. apply(), map()
df['Discounted'] = df['Price'].apply(lambda x: x*0.9)  # Apply to column
df['Category'] = df['Product'].map({'A':'X','B':'Y','C':'X','D':'Y'})  # Map values

# 3. replace()
df['Category'] = df['Category'].replace({'X':'Type1','Y':'Type2'})

# 4. Binning data
df['PriceRange'] = pd.cut(df['Price'], bins=[0, 100, 200, 500], labels=['Low', 'Medium', 'High'])

# 5. Categorical handling
df['Category'] = df['Category'].astype('category')

### **3.3.3 Date and Time in DataFrames**

In [33]:
data = {
    'event': ['Event A', 'Event B', 'Event C', 'Event D'],
    'date_str': ['2025-10-01', '2025-10-05', '2025-11-10', '2025-12-15'],
    'time_str': ['12:30:00', '15:45:00', '09:20:00', '18:00:00']
}
df = pd.DataFrame(data)
print("Original DataFrame:\n", df)

# ------------------------------
# 2. Converting strings to datetime
# ------------------------------
df['date'] = pd.to_datetime(df['date_str'])
df['datetime'] = pd.to_datetime(df['date_str'] + ' ' + df['time_str'])
print("\nDataFrame with datetime:\n", df)

# ------------------------------
# 3. Extracting components
# ------------------------------
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second
print("\nExtracted date components:\n", df)

# ------------------------------
# 4. Date arithmetic
# ------------------------------
df['next_day'] = df['date'] + pd.Timedelta(days=1)
df['prev_week'] = df['date'] - pd.Timedelta(weeks=1)
df['plus_hours'] = df['datetime'] + pd.Timedelta(hours=5)
print("\nDate arithmetic:\n", df)

# ------------------------------
# 5. Filtering by date
# ------------------------------
# Filter events after 2025-11-01
filtered_df = df[df['date'] > '2025-11-01']
print("\nFiltered events after 2025-11-01:\n", filtered_df)

# ------------------------------
# 6. Date ranges
# ------------------------------
date_range = pd.date_range(start='2025-10-01', end='2025-10-10', freq='D')
print("\nDate range from 2025-10-01 to 2025-10-10:\n", date_range)

# ------------------------------
# 7. Resampling and time series
# ------------------------------
# Creating a time series
ts_data = pd.Series(np.random.randn(10), 
                    index=pd.date_range('2025-10-01', periods=10, freq='D'))
print("\nTime series data:\n", ts_data)

# Resample to weekly sums
weekly_sum = ts_data.resample('W').sum()
print("\nWeekly sum:\n", weekly_sum)

# ------------------------------
# 8. Handling missing dates
# ------------------------------
ts_data_with_missing = ts_data.copy()
ts_data_with_missing = ts_data_with_missing.drop(ts_data_with_missing.index[3])
print("\nTime series with missing date:\n", ts_data_with_missing)

# Filling missing dates
ts_filled = ts_data_with_missing.asfreq('D', fill_value=0)
print("\nFilled missing dates:\n", ts_filled)

# ------------------------------
# 9. Date formatting
# ------------------------------
df['formatted_date'] = df['date'].dt.strftime('%d-%b-%Y')
df['formatted_datetime'] = df['datetime'].dt.strftime('%Y/%m/%d %H:%M:%S')
print("\nFormatted dates:\n", df[['formatted_date', 'formatted_datetime']])

# ------------------------------
# 10. Working with periods
# ------------------------------
period = pd.Period('2025-10', freq='M')  # Month period
print("\nMonthly period:", period)
print("Start of period:", period.start_time)
print("End of period:", period.end_time)

Original DataFrame:
      event    date_str  time_str
0  Event A  2025-10-01  12:30:00
1  Event B  2025-10-05  15:45:00
2  Event C  2025-11-10  09:20:00
3  Event D  2025-12-15  18:00:00

DataFrame with datetime:
      event    date_str  time_str       date            datetime
0  Event A  2025-10-01  12:30:00 2025-10-01 2025-10-01 12:30:00
1  Event B  2025-10-05  15:45:00 2025-10-05 2025-10-05 15:45:00
2  Event C  2025-11-10  09:20:00 2025-11-10 2025-11-10 09:20:00
3  Event D  2025-12-15  18:00:00 2025-12-15 2025-12-15 18:00:00

Extracted date components:
      event    date_str  time_str       date            datetime  year  month  \
0  Event A  2025-10-01  12:30:00 2025-10-01 2025-10-01 12:30:00  2025     10   
1  Event B  2025-10-05  15:45:00 2025-10-05 2025-10-05 15:45:00  2025     10   
2  Event C  2025-11-10  09:20:00 2025-11-10 2025-11-10 09:20:00  2025     11   
3  Event D  2025-12-15  18:00:00 2025-12-15 2025-12-15 18:00:00  2025     12   

   day    weekday  hour  minute  seco

## 3.4 Homework: Working with Pandas DataFrames in Data Science

### Task overview
In this assignment, you will practice working with **Pandas DataFrames** to perform essential data analysis and preprocessing tasks. You will simulate a small part of a **sensor data analytics pipeline**, a common workflow in data science used to clean, transform, and analyze time-series measurements collected from multiple sensors.  

### Your Task
You are given a dataset stored in **`data/sensor_data.csv`**, containing measurements from multiple sensors, including temperature and humidity readings at various timestamps.

### 1. **Load the Data**
- Load the CSV file into a Pandas DataFrame named `df`.
- Convert columns to appropriate data types (e.g., numeric, datetime, categorical).
- Set a **MultiIndex** using the combination of `sensor_id` and `timestamp`.  
  *Hint:* See [Pandas MultiIndex documentation](https://pandas.pydata.org/docs/user_guide/advanced.html).

### 2. **Data Cleaning**
- Remove all rows where:
  - `temperature` is **below the 20th percentile** or **above the 80th percentile**, **and**
  - any column has a missing value.  
  These rows should be treated as *outliers*.
- Report **how many rows were removed**.

### 3. **Filtering and Feature Engineering**
Perform the following operations on the cleaned DataFrame:

1. Count how many observations have:
   - `temperature` > 25, **and**
   - `humidity` < 40.
2. Create a new column that classifies each observation based on `temperature`:
   - `"low"` if below 20°C  
   - `"medium"` if between 20°C and 30°C  
   - `"high"` if above 30°C
3. Drop all rows where:
   - `sensor_id` == `"S5"`, **and**
   - `temperature` lies **outside one standard deviation** from the **global (overall)** mean temperature.

### 4. **Time and Date Analysis**
Add the following time-based columns:

1. **`hours_from_start`** – the difference in hours between each `timestamp` and the first timestamp in the dataset.
2. **`is_daytime`** – `True` if the measurement was taken between **6:00 AM and 6:00 PM**, otherwise `False`
3. **`after_12h`** – `True` if the measurement occurred **at least 12 hours after the first measurement** (you may optionally compute this per sensor).  
4. For **`sensor_id == "S2"`**, perform the following:
   - Create a **rolling average** of `temperature` over a **30-minute window**.
   - Compute the **difference** between the current and next `humidity` reading.
   - Create a Boolean column indicating whether:
     - `temperature` has increased by **more than 5°C**, and  
     - `humidity` has dropped by **more than 20** compared to the previous measurement.  
   - Save this subset of data to **`data/sensor_data_S2.csv`**.
5. **`is_weekend`** – indicate whether the measurement was taken on a **Saturday or Sunday**.

### Hints
- When computing statistics along columns, use `axis=0`.
- Prefer **vectorized operations** over `for` loops for efficiency.
- Add **comments or docstrings** to explain key parts of your code.


In [34]:
print("1.")
df=pd.read_csv('data/sensor_data.csv')
print("Shape:\n",df.shape,"\n", "Column indices:\n",df.columns,"\n","Type:\n", df.dtypes,"\n")  #data exploration
df['timestamp'] = pd.to_datetime(df['timestamp']) #turns elements of timestamp into date time type
df['sensor_id'] = df['sensor_id'].astype('category') #turns elements of sensor id into date categorical type
df['temperature'] = pd.to_numeric(df['temperature']) #turns elements of temperature into numerical type(already is, but for certainty)
df['humidity'] = pd.to_numeric(df['humidity']) #turns elements of humidity into numerical type(already is, but for certainty)
print("Type:\n", df.dtypes,"\n")
multiindex=pd.MultiIndex.from_frame(df[['sensor_id','timestamp']]) #makes multiindex from sensor id and timestamp
df.set_index(multiindex, inplace=True) #sets the index of dataframe to multiindex
df

1.
Shape:
 (1250, 4) 
 Column indices:
 Index(['timestamp', 'sensor_id', 'temperature', 'humidity'], dtype='object') 
 Type:
 timestamp       object
sensor_id       object
temperature    float64
humidity       float64
dtype: object 

Type:
 timestamp      datetime64[ns]
sensor_id            category
temperature           float64
humidity              float64
dtype: object 



timestamp sensor_id  temperature  \
sensor_id timestamp                                                        
S1        2025-10-30 08:00:00 2025-10-30 08:00:00        S1        22.51   
          2025-10-30 08:05:00 2025-10-30 08:05:00        S1        22.13   
          2025-10-30 08:10:00 2025-10-30 08:10:00        S1        22.63   
          2025-10-30 08:15:00 2025-10-30 08:15:00        S1        22.20   
          2025-10-30 08:20:00 2025-10-30 08:20:00        S1        22.66   
...                                           ...       ...          ...   
S5        2025-10-31 04:25:00 2025-10-31 04:25:00        S5        23.77   
          2025-10-31 04:30:00 2025-10-31 04:30:00        S5        23.65   
          2025-10-31 04:35:00 2025-10-31 04:35:00        S5        23.22   
          2025-10-31 04:40:00 2025-10-31 04:40:00        S5        23.40   
          2025-10-31 04:45:00 2025-10-31 04:45:00        S5        23.91   

                               humidity  
sensor_id timestamp                      
S1        2025-10-30 08:00:00     46.67  
          2025-10-30 08:05:00     43.08  
          2025-10-30 08:10:00     43.77  
          2025-10-30 08:15:00     44.36  
          2025-10-30 08:20:00     47.45  
...                                 ...  
S5        2025-10-31 04:25:00     48.94  
          2025-10-31 04:30:00     49.85  
          2025-10-31 04:35:00     48.78  
          2025-10-31 04:40:00     50.39  
          2025-10-31 04:45:00     47.91  

[1250 rows x 4 columns]

In [35]:
print("2.")
dfrmna=df.dropna() #removes rows with nan

per20=np.percentile(dfrmna['temperature'], 20,axis=0) #counts 20th percentile of temperature
per80=np.percentile(dfrmna['temperature'], 80, axis=0) #counts 80th percentile of temperature
above20 = dfrmna[dfrmna['temperature'] > per20] #selects rows above 20th percentile of temperature
cleandf = above20[above20['temperature'] < per80].copy() #selects rows under 80th percentile of temperature
print("Number of removed rows:",df.shape[0]-cleandf.shape[0])


2.
Number of removed rows: 507


In [36]:
print("3.1")
tunder20 = cleandf[cleandf['temperature'] > 25] #selects rows with temperature above 25
hover40 = tunder20[tunder20['humidity'] < 40] #selects rows with humidity less than 40
print("Number of data with temperature above 25:",tunder20.shape[0],"\n","Number of data with temperature above 25 and humidity under 40:",hover40.shape[0])

3.1
Number of data with temperature above 25: 0 
 Number of data with temperature above 25 and humidity under 40: 0


In [37]:
print("3.2")
np.min(cleandf['temperature'])
np.max(cleandf['temperature'])
cleandf['temperatureRange'] = pd.cut(cleandf['temperature'], bins=[0,20, 30,40], labels=['Low', 'Medium', 'High']) #makes a new column that bins elements of temperature into three bins(categories)
print("New column:","\n",cleandf.columns,"\n","Type:\n", cleandf.dtypes,"\n")
cleandf['temperatureRange']

3.2
New column: 
 Index(['timestamp', 'sensor_id', 'temperature', 'humidity',
       'temperatureRange'],
      dtype='object') 
 Type:
 timestamp           datetime64[ns]
sensor_id                 category
temperature                float64
humidity                   float64
temperatureRange          category
dtype: object 



sensor_id  timestamp          
S1         2025-10-30 08:00:00    Medium
           2025-10-30 08:10:00    Medium
           2025-10-30 08:20:00    Medium
           2025-10-30 08:45:00    Medium
           2025-10-30 09:00:00    Medium
                                   ...  
S5         2025-10-31 04:10:00    Medium
           2025-10-31 04:15:00    Medium
           2025-10-31 04:20:00    Medium
           2025-10-31 04:35:00    Medium
           2025-10-31 04:40:00    Medium
Name: temperatureRange, Length: 743, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

In [38]:
print("3.3")
withoutS5=cleandf[cleandf['sensor_id'] != 'S5'] #selects all data from sensor id 1 to 4
mu=cleandf['temperature'].mean(axis=0)
sd=cleandf['temperature'].std(axis=0)
print("Mean:",mu)
print("Standard deviation:",sd)
withoutoutsidesd=withoutS5[ (withoutS5['temperature'] >= mu - sd) &
    (withoutS5['temperature'] <= mu + sd)] # selects rows with temperature that are standard deviation under and above from mean
withoutoutsidesd

3.3
Mean: 22.902880215343203
Standard deviation: 0.31250409450506933


timestamp sensor_id  temperature  \
sensor_id timestamp                                                        
S1        2025-10-30 08:10:00 2025-10-30 08:10:00        S1        22.63   
          2025-10-30 08:20:00 2025-10-30 08:20:00        S1        22.66   
          2025-10-30 09:00:00 2025-10-30 09:00:00        S1        22.94   
          2025-10-30 09:15:00 2025-10-30 09:15:00        S1        23.00   
          2025-10-30 09:30:00 2025-10-30 09:30:00        S1        22.71   
...                                           ...       ...          ...   
S4        2025-10-31 03:50:00 2025-10-31 03:50:00        S4        22.75   
          2025-10-31 04:00:00 2025-10-31 04:00:00        S4        23.19   
          2025-10-31 04:05:00 2025-10-31 04:05:00        S4        22.97   
          2025-10-31 04:30:00 2025-10-31 04:30:00        S4        22.89   
          2025-10-31 04:45:00 2025-10-31 04:45:00        S4        22.95   

                               humidity temperatureRange  
sensor_id timestamp                                       
S1        2025-10-30 08:10:00     43.77           Medium  
          2025-10-30 08:20:00     47.45           Medium  
          2025-10-30 09:00:00     47.02           Medium  
          2025-10-30 09:15:00     47.37           Medium  
          2025-10-30 09:30:00     45.55           Medium  
...                                 ...              ...  
S4        2025-10-31 03:50:00     50.07           Medium  
          2025-10-31 04:00:00     47.35           Medium  
          2025-10-31 04:05:00     50.06           Medium  
          2025-10-31 04:30:00     47.50           Medium  
          2025-10-31 04:45:00     51.66           Medium  

[359 rows x 5 columns]

In [39]:
print("4.1")
start0=cleandf['timestamp'].min(axis=0) # finds a start
print(start0)
cleandf['HoursFromStart'] = (cleandf['timestamp'] - start0)/pd.Timedelta(hours=1) # makes a new column that counts the difference between timestamp and start in hours
cleandf['HoursFromStart']

4.1
2025-10-30 08:00:00


sensor_id  timestamp          
S1         2025-10-30 08:00:00     0.000000
           2025-10-30 08:10:00     0.166667
           2025-10-30 08:20:00     0.333333
           2025-10-30 08:45:00     0.750000
           2025-10-30 09:00:00     1.000000
                                    ...    
S5         2025-10-31 04:10:00    20.166667
           2025-10-31 04:15:00    20.250000
           2025-10-31 04:20:00    20.333333
           2025-10-31 04:35:00    20.583333
           2025-10-31 04:40:00    20.666667
Name: HoursFromStart, Length: 743, dtype: float64

In [40]:
print("4.2")
cleandf['is_daytime'] = cleandf['timestamp'].dt.hour.between(6, 17) #creates a new column with true if the timestamp is between 6 am and 6 pm
cleandf['is_daytime']

4.2


sensor_id  timestamp          
S1         2025-10-30 08:00:00     True
           2025-10-30 08:10:00     True
           2025-10-30 08:20:00     True
           2025-10-30 08:45:00     True
           2025-10-30 09:00:00     True
                                  ...  
S5         2025-10-31 04:10:00    False
           2025-10-31 04:15:00    False
           2025-10-31 04:20:00    False
           2025-10-31 04:35:00    False
           2025-10-31 04:40:00    False
Name: is_daytime, Length: 743, dtype: bool

In [41]:
print("4.3")
cleandf['after_12h'] = (cleandf['timestamp']-start0)>=pd.Timedelta(hours=12) #makes a new column with true if the difference between timestamp and start is greater than or equal to 12 hours
cleandf['after_12h']


4.3


sensor_id  timestamp          
S1         2025-10-30 08:00:00    False
           2025-10-30 08:10:00    False
           2025-10-30 08:20:00    False
           2025-10-30 08:45:00    False
           2025-10-30 09:00:00    False
                                  ...  
S5         2025-10-31 04:10:00     True
           2025-10-31 04:15:00     True
           2025-10-31 04:20:00     True
           2025-10-31 04:35:00     True
           2025-10-31 04:40:00     True
Name: after_12h, Length: 743, dtype: bool

In [42]:
print("4.4")

onlyS2 = cleandf[cleandf['sensor_id'] == 'S2'].copy()#selects data only with sensor id S2
onlyS2['tempRoll30'] = onlyS2.rolling('30min', on='timestamp')['temperature'].mean()#creates rolling average over 30 minute window
onlyS2['humidityNextDiff'] = onlyS2['humidity'].shift(-1) - onlyS2['humidity']# calculates the change in humidity

onlyS2['CritTempAndHum'] = (onlyS2['temperature'].diff() > 5) & (onlyS2['humidity'].diff() < -20) # creates new column that gives true if temperature increased more than 5°C and humidity dropped more then 20
onlyS2['CritTempAndHum']
onlyS2.to_csv('data/sensor_data_S2.csv')




4.4


In [43]:
print("4.5")
cleandf['is_weekend'] = cleandf['timestamp'].dt.weekday.between(1, 2) #creates new column that gives true if the timestamp is weekend day false if not
cleandf['is_weekend']

4.5


sensor_id  timestamp          
S1         2025-10-30 08:00:00    False
           2025-10-30 08:10:00    False
           2025-10-30 08:20:00    False
           2025-10-30 08:45:00    False
           2025-10-30 09:00:00    False
                                  ...  
S5         2025-10-31 04:10:00    False
           2025-10-31 04:15:00    False
           2025-10-31 04:20:00    False
           2025-10-31 04:35:00    False
           2025-10-31 04:40:00    False
Name: is_weekend, Length: 743, dtype: bool